In [9]:
import xml.etree.cElementTree as ET
import os
import re
import pandas as pd
import json
import random
random.seed(0)

import warnings
warnings.filterwarnings('ignore')

In [10]:
# tag_mapper = {'nu':'num', 'v':'vblex', 'cnj':'cnjcoo','3':'p3','2':'p2','1':'p1','':'None','s':'subj','p':'obj','a':'subj'}
tag_mapper = {'nu':'num', 'v':'vblex', 'cnj':'cnjcoo'}
NER_LIST = ['pn', 'on', 'fn', 'mn', 'sn', 'dn', 'gn', 'tn', 'wn', 'en', 'an', 'rn']

In [81]:
#for i in NER_LIST:
#    print(i,end = " ")

In [11]:
def txt_file_read(filename):
    lines=[]
    with open(filename, "r") as f:
        for line in f:
            line=line.strip()
            lines.append(line)
    return lines

In [12]:
def json_file_read(filename):
    with open(filename) as f:
        data = json.load(f)
    return data

In [13]:

# def process_tags(tags_string):
#     mapped_tags = []
#     org_tags = []
#     for tg in tags_string.split('.'):
#         l_tag = tg.lower().replace('3','p3').replace('2','p2').replace('1','p1')
#         # splitting on "-" as well
#         for t in l_tag.split('-'):
#             mp_tag = t
#             if t in tag_mapper.keys():
#                 mp_tag = tag_mapper[t]
#             mapped_tags.append(mp_tag)
#     return mapped_tags

In [14]:
def process_row(row):
    orignal = row[1]
    segm = row[2]
    lem = "_"
    eng_translation = "_"
    # match pattern - all groups starting with word and containing square bracket
    w_group = re.findall(r"[\'@\(\)a-zA-Z0-9_-]+\[\w+\]",segm)
    for w in w_group:
        start_index = w.find('[')
        end_index = w.find(']')
        lem = w[:start_index]
        eng_translation = w[start_index+1:end_index]
        break
    
    if eng_translation=='1':
        eng_translation = lem

    # process and save the tags 
    # org_tag_spl = re.split("\.|\-",row[3])
    org_tag_spl = re.split("\.",row[3])
    mp_tags_spl = []
    ner_tag=False
    for t in org_tag_spl:
        t=t.lower()
        mp_tag = t
        if t in tag_mapper.keys():
            mp_tag = tag_mapper[t]
        if t in NER_LIST:
            ner_tag=True
        mp_tags_spl.append(mp_tag)
    if ner_tag:
        mp_tags_spl.append('np')

    return lem,eng_translation,org_tag_spl, mp_tags_spl

In [15]:
# s = 'N.DEM2.ABS'
# org_tag_spl = re.split("\.|\-",s)
# org_tag_spl

# MTAAC GOLD CORPUS 

In [35]:
# define dataframe
column = ['filename','id','form','seg','xpostag','lem','eng_translation','org_tags_split', 'processed_tags_split']
extracted_df = pd.DataFrame(columns = column)
extracted_df.head()

,filename,id,form,seg,xpostag,lem,eng_translation,org_tags_split,processed_tags_split


In [36]:
base_dir = 'data/to_dict/'
conll_files = os.listdir('data/to_dict/')

In [10]:
for conll_file in conll_files:
    temp = os.path.join(base_dir,conll_file)
    f = txt_file_read(temp)
    f_lines = [line.split("\t") for line in f]

    # start processing file
    processed_ls = []
    for row in f_lines[2:]:
        if len(row)>3:
            lem,eng_translation,org_tag_spl, mp_tags_spl = process_row(row)
            processed_ls.append([conll_file, row[0], row[1], row[2], row[3], lem,eng_translation,org_tag_spl, mp_tags_spl])

        # print(orignal, lem,eng_translation,tags)
        # if orignal!="_" and lem!="_":
        
    processed_df = pd.DataFrame(processed_ls, columns=column)
    # print(processed_df)
    extracted_df = extracted_df.append(processed_df)

In [11]:
# for conll_file in conll_files:
#     temp = os.path.join(base_dir,conll_file)
#     f = txt_file_read(temp)
#     f_lines = [line.split("\t") for line in f]

#     # start processing file
#     processed_ls = []
#     for row in f_lines:
#         if len(row) > 3:
#             orignal, lem, eng_translation,org_tags, processed_tags = process_row(row)
#             # print(orignal, lem,eng_translation,tags)
#             if orignal!="_" and lem!="_":
#                 processed_ls.append([conll_file, row[0],orignal, lem,eng_translation,org_tags, processed_tags ])
        
#     processed_df = pd.DataFrame(processed_ls, columns=column)
#     # print(processed_df)
#     extracted_df = extracted_df.append(processed_df)

In [12]:
extracted_df

,filename,id,form,seg,xpostag,lem,eng_translation,org_tags_split,processed_tags_split
0,P100723.conll,o.1.1,1(disz),1(disz)[one],NU,1(disz),one,[NU],[num]
1,P100723.conll,o.1.2,udu-nita2,udunita[ram][-ø],N.ABS,udunita,ram,"[N, ABS]","[n, abs]"
2,P100723.conll,o.1.3,ba-usz2,ba-usz[die][-ø],MID.V.3-SG-S,ba-usz,die,"[MID, V, 3-SG-S]","[mid, vblex, 3-sg-s]"
3,P100723.conll,o.2.1,1(disz),1(disz)[one],NU,1(disz),one,[NU],[num]
4,P100723.conll,o.2.2,sila4,sila[lamb],N,sila,lamb,[N],[n]
...,...,...,...,...,...,...,...,...,...
27,P480072.conll,s1.col1.4.3,limmu2-ba,limmu[four]-bi-ak[-ø],NU.3-SG-NH-POSS.GEN.ABS,limmu,four,"[NU, 3-SG-NH-POSS, GEN, ABS]","[num, 3-sg-nh-poss, gen, abs]"
28,P480072.conll,s1.col2.1.1,a-a-kal-la,Ayakala[1],PN,Ayakala,Ayakala,[PN],"[pn, np]"
29,P480072.conll,s1.col2.2.1,ensi2,ensik[ruler],N,ensik,ruler,[N],[n]
30,P480072.conll,s1.col2.3.1,umma{ki},Umma[1][-ak][-ø],SN.GEN.ABS,Umma,Umma,"[SN, GEN, ABS]","[sn, gen, abs, np]"


In [13]:
extracted_df.to_csv(r'mtaac_vocab_all.txt', index=None, sep=' ', mode='w')

In [14]:
all_tags= set()
for i in extracted_df.processed_tags_split:
    for j in i:
        all_tags.add(j)

In [15]:
all_tags

{'',
 '1-sg-poss',
 '2-sg-a',
 '2-sg-poss',
 '3-nh',
 '3-pl',
 '3-pl-dat',
 '3-pl-s',
 '3-sg-a',
 '3-sg-cop',
 '3-sg-dat',
 '3-sg-h',
 '3-sg-h-a',
 '3-sg-h-dat',
 '3-sg-h-poss',
 '3-sg-hn-p',
 '3-sg-nh',
 '3-sg-nh-a',
 '3-sg-nh-l3',
 '3-sg-nh-p',
 '3-sg-nh-poss',
 '3-sg-p',
 '3-sg-s',
 '_',
 'abl',
 'abs',
 'an',
 'ant',
 'cnjcoo',
 'com',
 'cop-3-pl',
 'cop-3-sg',
 'dah-h',
 'dat',
 'dat-h',
 'dat-nh',
 'dem2',
 'dn',
 'dub[tablet]-ak',
 'en',
 'erg',
 'f',
 'fin',
 'fn',
 'gen',
 'gen-abl',
 'gn',
 'l1',
 'l1-syn',
 'l2',
 'l2-nh',
 'l3',
 'l3-nh',
 'mid',
 'mn',
 'mod',
 'n',
 'neg',
 'nf',
 'np',
 'num',
 'on',
 'pf',
 'pl',
 'pn',
 'pt',
 'rdp',
 'rn',
 'sn',
 'sub',
 'term',
 'tn',
 'vblex',
 'ven',
 'wn'}

### all tags for sux_eng dict

In [16]:
#Sample-  <sdef n="n" 	c="Noun"/>
for tg in all_tags:
    tg_str = f'''<sdef n="{tg}" c="{tg}"/>'''
    print(tg_str)

<sdef n="neg" c="neg"/>
<sdef n="en" c="en"/>
<sdef n="2-sg-a" c="2-sg-a"/>
<sdef n="rdp" c="rdp"/>
<sdef n="nf" c="nf"/>
<sdef n="" c=""/>
<sdef n="cop-3-pl" c="cop-3-pl"/>
<sdef n="3-sg-h-dat" c="3-sg-h-dat"/>
<sdef n="np" c="np"/>
<sdef n="3-sg-h-a" c="3-sg-h-a"/>
<sdef n="cop-3-sg" c="cop-3-sg"/>
<sdef n="1-sg-poss" c="1-sg-poss"/>
<sdef n="dat" c="dat"/>
<sdef n="pt" c="pt"/>
<sdef n="3-pl-dat" c="3-pl-dat"/>
<sdef n="2-sg-poss" c="2-sg-poss"/>
<sdef n="3-pl-s" c="3-pl-s"/>
<sdef n="fn" c="fn"/>
<sdef n="cnjcoo" c="cnjcoo"/>
<sdef n="l1-syn" c="l1-syn"/>
<sdef n="dah-h" c="dah-h"/>
<sdef n="on" c="on"/>
<sdef n="gen-abl" c="gen-abl"/>
<sdef n="sn" c="sn"/>
<sdef n="pn" c="pn"/>
<sdef n="tn" c="tn"/>
<sdef n="mid" c="mid"/>
<sdef n="f" c="f"/>
<sdef n="3-sg-hn-p" c="3-sg-hn-p"/>
<sdef n="l1" c="l1"/>
<sdef n="mod" c="mod"/>
<sdef n="3-sg-nh-l3" c="3-sg-nh-l3"/>
<sdef n="dat-nh" c="dat-nh"/>
<sdef n="erg" c="erg"/>
<sdef n="3-sg-nh-p" c="3-sg-nh-p"/>
<sdef n="3-nh" c="3-nh"/>
<sdef 

## Extracting from markdown devset christian

In [37]:
s = '''
s1.col1.1.1	{d}szu-{d}suen	Szusuen[1]	RN	_	_	_
s1.col1.2.1	lugal	lugal[king]	N	_	_	_
s1.col1.2.2	kal-ga	kalag[strong]-a	NF.V.PT	_	_	_
s1.col1.3.1	lugal	lugal[king]	N	_	_	_
s1.col1.3.2	uri5{ki}-ma	Urim[1]-ak	SN.GEN	_	_	_
s1.col1.4.1	lugal	lugal[king]	N	_	_	_
s1.col1.4.2	an-ub-da	anubda[quarter]	N	_	_	_
s1.col1.4.3	limmu2-ba	limmu[four]-bi-ak[-ø]	NU.3-SG-NH-POSS.GEN.ABS	_	_	_
1	{d}szara2	Szara[1][-ak]	DN
2	nir-gal2	_[jewel] 	N
3	an-na	An[1][-ak][-ak][sze]	DN.GEN
4   dumu    dumu[child] N
5   ki-ag2  ki'ag[love[-ø]  NF.V
1	{d}szara2	Szara[1][-ak]	DN
7	lugal-a-ni-ir	_[his.lord] N.3-SG-H-POSS.DAT-H
4   dumu    dumu[child] N
5   ki-ag2  ki'ag[love][-ø] NF.V 
6   {d}inanna   Inanna[1][-ak]  DN.GEN
1	{d}szara2	Szara[1][-ak]   DN
7	lugal-a-ni-ir	_[his.lord] N.3-SG-H-POSS.DAT-H
1	{d}szara2	Szara[1][-ak]	DN	5	DAT	_	
2	nir-gal2	_[jewel]	NF.V	5	_	_	
3	an-na	An[1][-ak][-ak][sze]	DN.GEN	2	GEN	_	
4	dumu    dumu[child] N   5	_	heuristic: before verbal head	
5	ki-ag2	ki'ag[love][-ø]	NF.V	0	root	_	
6	{d}inanna	Inanna[1][-ak]	DN	5	GEN	_	
7	lugal-a-ni-ir	_[his.lord]	N.3-SG-H-POSS.DAT-H	6	appos	_
8	nam-ti	namtil[life]    N	5	TERM	_	
9	{d}szu-{d}suen	Szusuen[1]  RN	8	GEN	_	
10	lugal	lugal[king] N	9	appos	_	
11	kal-ga	kalag[strong]-a NF.V.SUB	10	acl	_	
12	lugal	lugal[king] N	9	appos	_	
13	uri5{ki}-ma	Urim[1]-ak  SN.GEN	12	GEN	_	
14	lugal	lugal[king] N	9	appos	_	
15	an	An[1][-ra]  N	17	nummod	projectivity violation	
16	ub-da	_   N	9	appos	projectivity violation	
17	limmu2-ba-ka-sze3	_[quarters] NF.V.3-SG-NH-POSS.GEN.GEN.TERM	14	nummod	_
18	ur-{d}...	_[1][-ra]   PN.GEN	5	acl	_	
19	aga3-us2	aga'us[soldier] N	18	ABS	_	
20	lugal	lugal[king]	N	19	acl	_	
21	dumu	dumu[child]	N	19	appos	_	
22	ur-ab-ba-ke4	_[Ur-abba]  PN.GEN.ERG	21	GEN	_
17	ma2-dara3-abzu	Madara'abzu[1]	ON
18	{d}en-ki-ka	Enkik[1][-ak]	DN.GEN.ABS
2	mu	mu[year]	N
3	didli	didli[several][-ø][-ak] NF.V.ABS
1	pisan-dub-ba	bisagdubak[filing_basket]	N
2	mu	mu[year]	N
3	didli	didli[several][-ø][-ak]	NF.V.ABS
4	sze-ba	szeba[ration]	N.ABS
5	sze	sze[barley]	N
6	erin2-na	erin[people]	N.GEN
8	sza3-bi	szag[heart]-bi[-ta]	N.3-SG-NH-POSS.ABL
9	su-ga	sug[repay]-a    NF.V.PT
10	nig2-ka9	nigSZID[account]    N
11	sze-ka	_[of.barley]    N.GEN
1	pisan-dub-ba	bisagdubak[filing_basket]   N	0	root	_	
13	mu	mu[year]	N	12	parataxis	_	
14	{d}szu-{d}suen	Szusuen[1]	RN	13	appos	_	
15	lugal	lugal[king]	N	14	appos	_	
16	uri5{ki}-ma-ke4	Urim[1]-ak-e	SN.GEN.ERG	15	GEN	_	
17	ma2-dara3-abzu	Madara'abzu[1]	ON	15	ABS	_	
18	{d}en-ki-ka	Enkik[1][-ak]   DN.GEN.ABS	17	GEN	_	
19	bi2-in-du8	_[was.caulked.By]   3-SG-NH.L3.3-SG-H-A.V.3-SG-P	14	acl	_
'''

In [49]:
processed_ls = []
for row in s.split("\n"):
    row = row.split("\t")
    # print(row)
    if len(row)>3:
        lem,eng_translation,org_tag_spl, mp_tags_spl = process_row(row)
        # print(lem,eng_translation,org_tag_spl, mp_tags_spl)
        processed_ls.append(['_', row[0], row[1], row[2], row[3], lem,eng_translation,org_tag_spl, mp_tags_spl])

    # print(orignal, lem,eng_translation,tags)
    # if orignal!="_" and lem!="_":
    
extracted_df = pd.DataFrame(processed_ls, columns=column)
# print(processed_df)
# extracted_df = extracted_df.append(processed_df)

In [51]:
extracted_df.head()

,filename,id,form,seg,xpostag,lem,eng_translation,org_tags_split,processed_tags_split
0,_,s1.col1.1.1,{d}szu-{d}suen,Szusuen[1],RN,Szusuen,Szusuen,[RN],"[rn, np]"
1,_,s1.col1.2.1,lugal,lugal[king],N,lugal,king,[N],[n]
2,_,s1.col1.2.2,kal-ga,kalag[strong]-a,NF.V.PT,kalag,strong,"[NF, V, PT]","[nf, vblex, pt]"
3,_,s1.col1.3.1,lugal,lugal[king],N,lugal,king,[N],[n]
4,_,s1.col1.3.2,uri5{ki}-ma,Urim[1]-ak,SN.GEN,Urim,Urim,"[SN, GEN]","[sn, gen, np]"


In [52]:
# processed_ls

## MAJOR POS TAGS
### np(Proper Name), num(numbers), cnjcoo, n, vblex

In [53]:
extracted_df.head(10)

,filename,id,form,seg,xpostag,lem,eng_translation,org_tags_split,processed_tags_split
0,_,s1.col1.1.1,{d}szu-{d}suen,Szusuen[1],RN,Szusuen,Szusuen,[RN],"[rn, np]"
1,_,s1.col1.2.1,lugal,lugal[king],N,lugal,king,[N],[n]
2,_,s1.col1.2.2,kal-ga,kalag[strong]-a,NF.V.PT,kalag,strong,"[NF, V, PT]","[nf, vblex, pt]"
3,_,s1.col1.3.1,lugal,lugal[king],N,lugal,king,[N],[n]
4,_,s1.col1.3.2,uri5{ki}-ma,Urim[1]-ak,SN.GEN,Urim,Urim,"[SN, GEN]","[sn, gen, np]"
5,_,s1.col1.4.1,lugal,lugal[king],N,lugal,king,[N],[n]
6,_,s1.col1.4.2,an-ub-da,anubda[quarter],N,anubda,quarter,[N],[n]
7,_,s1.col1.4.3,limmu2-ba,limmu[four]-bi-ak[-ø],NU.3-SG-NH-POSS.GEN.ABS,limmu,four,"[NU, 3-SG-NH-POSS, GEN, ABS]","[num, 3-sg-nh-poss, gen, abs]"
8,_,1,{d}szara2,Szara[1][-ak],DN,Szara,Szara,[DN],"[dn, np]"
9,_,2,nir-gal2,_[jewel],N,_,jewel,[N],[n]


# Named Entity

In [54]:
NP_sux = set()
NP_dict = set()
for i, row in extracted_df.iterrows():
    p_tag = row['processed_tags_split']
    #form = row['form']
    form = row['form'].replace(':','-').replace('{','\{').replace('}','\}').replace('[','\[').replace(']','\]').replace('(','\(').replace(')','\)').replace('|','\|') # to escape the special meaning of brackets
    lemma = row['lem']
    if 'np' in p_tag:
        remainig_tags = list(set(p_tag)-set(['np']))
        pos_tag = 'np'
        tg_str = '<np>'
        for t in remainig_tags:
            tg_str+='<'+t+'>'

        # for morph dict
        sux_morph = lemma+tg_str+':'+form

        # for translation dict
        sux_eng_dict = f'''<e><p><l>{lemma}<s n="{pos_tag}"/></l><r>{lemma}<s n="{pos_tag}"/></r></p></e>'''
        
        NP_sux.add(sux_morph)
        NP_dict.add(sux_eng_dict)

    

    # for j in NER_LIST:
    #     if j in p_tag:
    #         tag_found=True
    #         break
    # if tag_found:
    #     PN_LIST.add(row['form'])

In [55]:
for i in NP_sux:
    print(i)

Madara'abzu<np><on>:ma2-dara3-abzu
Inanna<np><dn>:\{d\}inanna
Enkik<np><abs><gen><dn>:\{d\}en-ki-ka
An<np><gen><dn>:an-na
Szara<np><dn>:\{d\}szara2
Urim<np><gen><sn>:uri5\{ki\}-ma
Szusuen<np><rn>:\{d\}szu-\{d\}suen
Urim<np><erg><gen><sn>:uri5\{ki\}-ma-ke4


In [56]:
for i in NP_dict:
    print(i)

<e><p><l>Szara<s n="np"/></l><r>Szara<s n="np"/></r></p></e>
<e><p><l>An<s n="np"/></l><r>An<s n="np"/></r></p></e>
<e><p><l>Enkik<s n="np"/></l><r>Enkik<s n="np"/></r></p></e>
<e><p><l>Szusuen<s n="np"/></l><r>Szusuen<s n="np"/></r></p></e>
<e><p><l>Urim<s n="np"/></l><r>Urim<s n="np"/></r></p></e>
<e><p><l>Inanna<s n="np"/></l><r>Inanna<s n="np"/></r></p></e>
<e><p><l>Madara'abzu<s n="np"/></l><r>Madara'abzu<s n="np"/></r></p></e>


In [57]:
# #sample - Dugrna<np>:du-gr\{d\}-na
# for tg in PN_LIST:
#     original = tg.replace('{','\{').replace('}','\}')
#     formatted_1 = re.sub('\{[a-z\.]*\}','',tg)
#     formatted_2 = re.sub('[0-9]+','',formatted_1)
#     formatted_3 = formatted_2.replace('-','').capitalize()
#     print(formatted_3+'<np>:'+original)

In [58]:
# # to generate for dict
# # <e><p><l>Inanna<s n="np"/></l><r>Inanna<s n="np"/></r></p></e>
# for tg in PN_LIST:
#     original = tg.replace('{','\{').replace('}','\}')
#     formatted_1 = re.sub('\{[a-z\.]*\}','',tg)
#     formatted_2 = re.sub('[0-9]+','',formatted_1)
#     formatted_3 = formatted_2.replace('-','').capitalize()
#     print(f'''<e><p><l>{formatted_3}<s n="np"/></l><r>{formatted_3}<s n="np"/></r></p></e>''')

# Numbers

In [59]:
NP_sux = set()
NP_dict = set()
for i, row in extracted_df.iterrows():
    p_tag = row['processed_tags_split']
    form = row['form'].replace(':','-').replace('{','\{').replace('}','\}').replace('[','\[').replace(']','\]').replace('(','\(').replace(')','\)').replace('|','\|') # to escape the special meaning of brackets
    # form = row['form'].replace('{','\{').replace('}','\}').replace('[','\[').replace(']','\]') # to escape the special meaning of brackets
    lemma = row['lem']
    eng_translation = row['eng_translation']
    if 'num' in p_tag:
        remainig_tags = list(set(p_tag)-set(['num']))
        pos_tag = 'num'
        tg_str = '<num>'
        for t in remainig_tags:
            tg_str+='<'+t+'>'

        # for morph dict
        sux_morph = lemma+tg_str+':'+form

        # for translation dict
        sux_eng_dict = f'''<e><p><l>{lemma}<s n="{pos_tag}"/></l><r>{eng_translation}<s n="{pos_tag}"/></r></p></e>'''
        
        NP_sux.add(sux_morph)
        NP_dict.add(sux_eng_dict)

    

    # for j in NER_LIST:
    #     if j in p_tag:
    #         tag_found=True
    #         break
    # if tag_found:     
    #     PN_LIST.add(row['form'])

In [60]:
for i in NP_sux:
    print(i)

limmu<num><3-sg-nh-poss><abs><gen>:limmu2-ba


In [61]:
for i in NP_dict:
    print(i)

<e><p><l>limmu<s n="num"/></l><r>four<s n="num"/></r></p></e>


# Cnjcoo

In [62]:
NP_sux = set()
NP_dict = set()
for i, row in extracted_df.iterrows():
    p_tag = row['processed_tags_split']
    form = row['form'].replace(':','-').replace('{','\{').replace('}','\}').replace('[','\[').replace(']','\]').replace('(','\(').replace(')','\)').replace('|','\|') # to escape the special meaning of brackets
    # form = row['form'].replace('{','\{').replace('}','\}').replace('[','\[').replace(']','\]') # to escape the special meaning of brackets
    lemma = row['lem']
    eng_translation = row['eng_translation']
    if 'cnjcoo' in p_tag:
        remainig_tags = list(set(p_tag)-set(['cnjcoo']))
        pos_tag = 'cnjcoo'
        tg_str = '<cnjcoo>'
        for t in remainig_tags:
            tg_str+='<'+t+'>'

        # for morph dict
        sux_morph = lemma+tg_str+':'+form

        # for translation dict
        sux_eng_dict = f'''<e><p><l>{lemma}<s n="{pos_tag}"/></l><r>{eng_translation}<s n="{pos_tag}"/></r></p></e>'''
        
        NP_sux.add(sux_morph)
        NP_dict.add(sux_eng_dict)

    

    # for j in NER_LIST:
    #     if j in p_tag:
    #         tag_found=True
    #         break
    # if tag_found:
    #     PN_LIST.add(row['form'])

In [63]:
for i in NP_sux:
    print(i)

In [64]:
for i in NP_dict:
    print(i)

# Noun

In [66]:
NP_sux = set()
NP_dict = set()
for i, row in extracted_df.iterrows():
    p_tag = row['processed_tags_split']
    form = row['form'].replace(':','-').replace('{','\{').replace('}','\}').replace('[','\[').replace(']','\]').replace('(','\(').replace(')','\)').replace('|','\|') # to escape the special meaning of brackets
    # form = row['form'].replace('{','\{').replace('}','\}').replace('[','\[').replace(']','\]') # to escape the special meaning of brackets
    # lemma = row['lem']
    lemma = re.sub('[^a-zA-Z0-9]+','_',row['form'])

    eng_translation = row['eng_translation']
    if 'n' in p_tag:
        remainig_tags = list(set(p_tag)-set(['n']))
        pos_tag = 'n'
        tg_str = '<n>'
        for t in remainig_tags:
            tg_str+='<'+t+'>'

        # for morph dict
        sux_morph = lemma+tg_str+':'+form

        # for translation dict
        sux_eng_dict = f'''<e><p><l>{lemma}<s n="{pos_tag}"/></l><r>{eng_translation}<s n="{pos_tag}"/></r></p></e>'''
        
        NP_sux.add(sux_morph)
        NP_dict.add(sux_eng_dict)

    

    # for j in NER_LIST:
    #     if j in p_tag:
    #         tag_found=True
    #         break
    # if tag_found:
    #     PN_LIST.add(row['form'])

In [67]:
for i in NP_sux:
    print(i)

erin2_na<n><gen>:erin2-na
sza3_bi<n><3-sg-nh-poss><abl>:sza3-bi
sze<n>:sze
nir_gal2<n>:nir-gal2
lugal<n>:lugal
dumu<n>:dumu
an_ub_da<n>:an-ub-da
pisan_dub_ba<n>:pisan-dub-ba
mu<n>:mu
sze_ba<n><abs>:sze-ba
lugal_a_ni_ir<n><3-sg-h-poss><dat-h>:lugal-a-ni-ir


In [69]:
for i in NP_dict:
    print(i)

<e><p><l>sza3_bi<s n="n"/></l><r>heart<s n="n"/></r></p></e>
<e><p><l>lugal<s n="n"/></l><r>king<s n="n"/></r></p></e>
<e><p><l>pisan_dub_ba<s n="n"/></l><r>filing_basket<s n="n"/></r></p></e>
<e><p><l>sze<s n="n"/></l><r>barley<s n="n"/></r></p></e>
<e><p><l>mu<s n="n"/></l><r>year<s n="n"/></r></p></e>
<e><p><l>dumu<s n="n"/></l><r>child<s n="n"/></r></p></e>
<e><p><l>nir_gal2<s n="n"/></l><r>jewel<s n="n"/></r></p></e>
<e><p><l>sze_ba<s n="n"/></l><r>ration<s n="n"/></r></p></e>
<e><p><l>erin2_na<s n="n"/></l><r>people<s n="n"/></r></p></e>
<e><p><l>lugal_a_ni_ir<s n="n"/></l><r>_<s n="n"/></r></p></e>
<e><p><l>an_ub_da<s n="n"/></l><r>quarter<s n="n"/></r></p></e>


In [55]:
# OLD Nouns - 
# lugal<n>:lugal
# digir<n>:digir
# ama<n>:ama
# nin<n>:nin
# egal<n>:e2-gal
# dubsar<n>:dub-sar
# ursag<n>:ur-sag
# e2<n>:e2
# udu<n>:udu
# ur<n>:ur
# na<n>:na<4>
# dub-sar<n>:dub-sar
# ki<n>:ki
# di-ku5<n>:di-ku5
# ezem<n>:ezem


# Verb

In [70]:
NP_sux = set()
NP_dict = set()
for i, row in extracted_df.iterrows():
    p_tag = row['processed_tags_split']
    form = row['form'].replace(':','-').replace('{','\{').replace('}','\}').replace('[','\[').replace(']','\]').replace('(','\(').replace(')','\)').replace('|','\|') # to escape the special meaning of brackets
    # form = row['form'].replace('{','\{').replace('}','\}').replace('[','\[').replace(']','\]') # to escape the special meaning of brackets
    # lemma = row['lem']
    lemma = re.sub('[^a-zA-Z0-9]+','_',row['form'])

    eng_translation = row['eng_translation']
    if 'vblex' in p_tag:
        remainig_tags = list(set(p_tag)-set(['vblex']))
        pos_tag = 'vblex'
        tg_str = '<vblex>'
        for t in remainig_tags:
            tg_str+='<'+t+'>'

        # for morph dict
        sux_morph = lemma+tg_str+':'+form

        # for translation dict
        sux_eng_dict = f'''<e><p><l>{lemma}<s n="{pos_tag}"/></l><r>{eng_translation}<s n="{pos_tag}"/></r></p></e>'''
        
        NP_sux.add(sux_morph)
        NP_dict.add(sux_eng_dict)

    

    # for j in NER_LIST:
    #     if j in p_tag:
    #         tag_found=True
    #         break
    # if tag_found:
    #     PN_LIST.add(row['form'])

In [71]:
for i in NP_sux:
    print(i)

kal_ga<vblex><pt><nf>:kal-ga
didli<vblex><abs><nf>:didli
ki_ag2<vblex><nf>:ki-ag2
nir_gal2<vblex><nf>:nir-gal2


In [72]:
for i in NP_dict:
    print(i)

<e><p><l>ki_ag2<s n="vblex"/></l><r>love<s n="vblex"/></r></p></e>
<e><p><l>nir_gal2<s n="vblex"/></l><r>jewel<s n="vblex"/></r></p></e>
<e><p><l>didli<s n="vblex"/></l><r>several<s n="vblex"/></r></p></e>
<e><p><l>kal_ga<s n="vblex"/></l><r>strong<s n="vblex"/></r></p></e>


## Extracting from Devgit

In [133]:
s ='''1	{d}szara2	Szara[1][-ak]	DN	DN
2	nir-gal2	_[jewel]	N 	N
3	an-na	An[1][-ak][-ak][sze]	PN	DN.GEN
4	dumu	dumu[child]	N	N
5	ki-ag2	ki'ag[love][-ø]	V	NF.V
6	{d}inanna	Inanna[1][-ak]	DN	DN.GEN
7	lugal-a-ni-ir	_[his.lord]	N	N.3-SG-H-POSS.DAT-H
8	nam-ti	namtil[life]	N	N
9	{d}szu-{d}suen	Szusuen[1]	RN	RN
10	lugal	lugal[king]	N	N
11	kal-ga	kalag[strong]-a	V	NF.V.SUB
12	lugal	lugal[king]	N	N
13	uri5{ki}-ma	Urim[1]-ak	SN	SN.GEN
14	lugal	lugal[king]	N	N
15	an-ub-da	[quarters]	N	N
16	limmu2-ba-ka-sze3	[four]-b-ak-ak-sze	NU	NF.V.3-SG-NH-POSS.GEN.GEN.TERM
17	ur-{d}...	_[1][-ra]	PN	PN
18	aga3-us2	aga'us[soldier]	N	N
19	lugal	lugal[king][-ak]	N	N.GEN
20	dumu	dumu[child]	N	N
21	ur-ab-ba-ke4	_[Ur-abba]	PN	PN.GEN.ERG
22	mu-na-dim2	mu-nn-a-dim[create][-ø]	V	VEN.3-SG-H.DAT.3-SG-H-A.V.3-SG-P''' 

In [134]:
sen = ""
for line in s.split('\n'):
    sen+=line.split("\t")[1]+" "

In [135]:
sen

'{d}szara2 nir-gal2 an-na dumu ki-ag2 {d}inanna lugal-a-ni-ir nam-ti {d}szu-{d}suen lugal kal-ga lugal uri5{ki}-ma lugal an-ub-da limmu2-ba-ka-sze3 ur-{d}... aga3-us2 lugal dumu ur-ab-ba-ke4 mu-na-dim2 '

In [40]:
import onmt

/opt/homebrew/anaconda3/envs/gsoc/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/opt/homebrew/anaconda3/envs/gsoc/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: (__ZN3c106SymInt8toSymIntENS_13intrusive_ptrINS_14SymIntNodeImplENS_6detail34intrusive_target_default_null_typeIS2_EEEE)
  Referenced from: '/opt/homebrew/anaconda3/envs/gsoc/lib/python3.10/site-packages/torchvision/image.so'
  Expected in: '/opt/homebrew/anaconda3/envs/gsoc/lib/python3.10/site-packages/torch/lib/libc10.dylib'
  warn(f"Failed to load image Python extension: {e}")


OSError: dlopen(/opt/homebrew/anaconda3/envs/gsoc/lib/python3.10/site-packages/torchaudio/lib/libtorchaudio.so, 0x0006): Symbol not found: (__ZN2at27getStepCallbacksUnlessEmptyENS_11RecordScopeE)
  Referenced from: '/opt/homebrew/anaconda3/envs/gsoc/lib/python3.10/site-packages/torchaudio/lib/libtorchaudio.so'
  Expected in: '/opt/homebrew/anaconda3/envs/gsoc/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib'

## error correction GCE

In [19]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

/opt/homebrew/anaconda3/envs/gsoc/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
model = T5ForConditionalGeneration.from_pretrained("Unbabel/gec-t5_small")
tokenizer = T5Tokenizer.from_pretrained('t5-small')
# tokenizer = AutoTokenizer.from_pretrained("SafiUllahShahid/EnGECmodel")
# model = AutoModelForSeq2SeqLM.from_pretrained("SafiUllahShahid/EnGECmodel")
# tokenizer = AutoTokenizer.from_pretrained("juancavallotti/t5-base-gec")
# model = AutoModelForSeq2SeqLM.from_pretrained("juancavallotti/t5-base-gec")

# tokenizer = AutoTokenizer.from_pretrained("vennify/t5-base-grammar-correction")
# model = AutoModelForSeq2SeqLM.from_pretrained("vennify/t5-base-grammar-correction")

/opt/homebrew/anaconda3/envs/gsoc/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:164: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [21]:
#sentence = "I like to swimming"

In [22]:
s = '''repaid #from its  heart   account #of  barley'''

In [23]:
sentence = " ".join(s.replace('#','').strip().split())
sentence

'repaid from its heart account of barley'

## Tokenization

In [25]:
tokenized_sentence = tokenizer.tokenize(sentence)
tokenized_sentence

['▁',
 're',
 'paid',
 '▁from',
 '▁its',
 '▁heart',
 '▁account',
 '▁of',
 '▁bar',
 'ley']

In [26]:
tokenIds = tokenizer.convert_tokens_to_ids(tokenized_sentence)
tokenIds

[3, 60, 12760, 45, 165, 842, 905, 13, 1207, 1306]

In [31]:
final_input = tokenizer.prepare_for_model(tokenIds)
final_input

{'input_ids': [3, 60, 12760, 45, 165, 842, 905, 13, 1207, 1306, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [32]:
tokenizer.decode(final_input['input_ids'])

'repaid from its heart account of barley</s>'

In [38]:
tokenized_sentence = tokenizer(sentence, max_length=20, truncation=True, padding='max_length', return_tensors='pt')

In [39]:
tokenized_sentence

{'input_ids': tensor([[    3,    60, 12760,    45,   165,   842,   905,    13,  1207,  1306,
             1,     0,     0,     0,     0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}

In [187]:
corrected_sentence = tokenizer.decode(
    model.generate(
        input_ids = tokenized_sentence.input_ids,
        attention_mask = tokenized_sentence.attention_mask, 
        max_length=512, 
        num_beams=5,
        early_stopping=True,
    )[0],
    skip_special_tokens=True, 
    clean_up_tokenization_spaces=True
)

In [183]:
corrected_sentence

'repaid from its heart account of barley'

In [11]:
# from transformers import AutoTokenizer, AutoModelForTokenClassification
# from transformers import AutoTokenizer, AutoModelForTokenClassification

# tokenizer = AutoTokenizer.from_pretrained("felflare/bert-restore-punctuation")

# model = AutoModelForTokenClassification.from_pretrained("felflare/bert-restore-punctuation")

In [12]:
# from transformers import AutoTokenizer, AutoModelForTokenClassification

# tokenizer = AutoTokenizer.from_pretrained("st1992/bert-restore-punctuation")

# model = AutoModelForTokenClassification.from_pretrained("st1992/bert-restore-punctuation")

In [8]:
# import sys
# sys.path
# sys.path.append('/Users/himanshu/GSOC/gingerit')

In [9]:
# from gingerit.gingerit import GingerIt

In [10]:
# parser = GingerIt()

# text = "repaid from its heart account of barley"

# print(parser.parse(text)['result'])

In [13]:
# sentence = "repaid from its heart account of barley"

In [14]:
# tokenized_sentence = tokenizer(sentence, max_length=513, truncation=True, padding='max_length', return_tensors='pt')

In [15]:
# corrected_sentence = tokenizer.decode(
#     model.generate(
#         input_ids = tokenized_sentence.input_ids,
#         attention_mask = tokenized_sentence.attention_mask, 
#         max_length=513,
#         num_beams=5,
#         early_stopping=True,
#     )[0],
#     skip_special_tokens=True, 
#     clean_up_tokenization_spaces=True
# )

# Extra

In [79]:
import re
s = '{d}nin-mar{ki}-ta'

In [80]:
re.sub('\{[a-z\.]*\}','',s)

'nin-mar-ta'

In [81]:
re.findall(r'\{.*\}',s)

['{d}nin-mar{ki}']

In [112]:
extracted_df.head()

,filename,id,form,lem,eng_translation,org_tags,processed_tags
0,P100723.conll,o.1.1,1(disz),1(disz),one,NU,[num]
1,P100723.conll,o.1.2,udu-nita2,udunita,ram,N.ABS,"[n, abs]"
2,P100723.conll,o.1.3,ba-usz2,ba-usz,die,MID.V.3-SG-S,"[mid, vblex, p3, sg, s]"
3,P100723.conll,o.2.1,1(disz),1(disz),one,NU,[num]
4,P100723.conll,o.2.2,sila4,sila,lamb,N,[n]


In [23]:
extracted_df

,filename,id,form,lem,eng_translation,org_tags,processed_tags
0,P100723.conll,o.1.1,1(disz),1(disz),one,NU,[num]
1,P100723.conll,o.1.2,udu-nita2,udunita,ram,N.ABS,"[n, abs]"
2,P100723.conll,o.1.3,ba-usz2,ba-usz,die,MID.V.3-SG-S,"[mid, vblex, p3, sg, s]"
3,P100723.conll,o.2.1,1(disz),1(disz),one,NU,[num]
4,P100723.conll,o.2.2,sila4,sila,lamb,N,[n]
...,...,...,...,...,...,...,...
27,P480072.conll,s1.col1.4.3,limmu2-ba,limmu,four,NU.3-SG-NH-POSS.GEN.ABS,"[num, p3, sg, nh, poss, gen, abs]"
28,P480072.conll,s1.col2.1.1,a-a-kal-la,Ayakala,1,PN,[pn]
29,P480072.conll,s1.col2.2.1,ensi2,ensik,ruler,N,[n]
30,P480072.conll,s1.col2.3.1,umma{ki},Umma,1,SN.GEN.ABS,"[sn, gen, abs]"


In [26]:
# extracted_df.to_csv('vocab.csv')                                            

In [27]:
# extracted_df.to_csv(r'vocab_form.txt', header=None, index=None, sep=' ', mode='a')

In [332]:
# converting all proper noun (FN, DN, PN) etc to there names
for i, row in extracted_df.iterrows():
    if row['eng_translation']=='1':
        row['eng_translation'] = row['lem']

In [392]:
extracted_df[['form','lem','eng_translation']]

,form,lem,eng_translation
0,1(disz),1(disz),one
1,udu-nita2,udunita,ram
2,ba-usz2,ba-usz,die
3,1(disz),1(disz),one
4,sila4,sila,lamb
...,...,...,...
27,limmu2-ba,limmu,four
28,a-a-kal-la,Ayakala,1
29,ensi2,ensik,ruler
30,umma{ki},Umma,1


## Alphabets

In [333]:
alpha = set()
for word in extracted_df['form']:
    for char in word:
        alpha.add(char)
alpha = sorted(alpha)
print(len(alpha))
alphs_string = "".join(alpha)
alphs_string

66


"#'()-./0123456789:<>?@ABDEGIKLMNPRSTUWZ[]abcdeghiklmnpqrstuvwxz{|}"

### Add alphabets to dict

In [334]:
base_dix_path = './apertium-sux.sux.dix'
mytree = ET.parse(base_dix_path)
root = mytree.getroot()
root

<Element 'dictionary' at 0x13b021bc0>

In [335]:
old_alpha = root.find('alphabet')
root.remove(old_alpha)

In [336]:
el_alpha = ET.Element("alphabet")
el_alpha.text = alphs_string

In [337]:
root.append(el_alpha)

In [338]:
tree = ET.ElementTree(root)
# ET.indent(tree, space="\t", level=0)

In [339]:
sux_dix_name = "apertium-sux.sux.dix"
with open (sux_dix_name, "wb") as f :
        tree.write(f,encoding = "UTF-8", xml_declaration = True)

## unique tags

In [340]:
uniqe_tags = set()
for tg in extracted_df['tags']:
    for t in tg:
        if t not in ['','_']:
            uniqe_tags.add(t)
uniqe_tags = sorted(uniqe_tags)
uniqe_tags[:5]

['a', 'abl', 'abs', 'an', 'ant']

In [341]:
len(uniqe_tags)

53

### mappings (once get the names)

In [342]:
# read json
filepath = '../Data/CDLI-CoNLL-to-CoNLLU-Converter/cdliconll2conllu/mapping.json'
cdliconll_map_data = json_file_read(filepath)

In [343]:
cdliconll_map_data

{'CDLI-CoNLL-Fields': ['ID',
  'FORM',
  'SEGM',
  'XPOSTAG',
  'HEAD',
  'DEPREL',
  'MISC'],
 'CoNLL-U-Fields': ['ID',
  'FORM',
  'LEMMA',
  'UPOSTAG',
  'XPOSTAG',
  'FEATS',
  'HEAD',
  'DEPREL',
  'DEPS',
  'MISC'],
 'xPOSTagMapping': {'AJ': 'ADJ',
  'AV': 'ADV',
  'NU': 'NUM',
  'CNJ': 'CCONJ',
  'DET': 'DET',
  'J': 'INTJ',
  'N': 'NOUN',
  'V': 'VERB',
  'PRP': 'ADP',
  'DN': 'PROPN',
  'EN': 'PROPN',
  'GN': 'PROPN',
  'MN': 'PROPN',
  'PN': 'PROPN',
  'RN': 'PROPN',
  'SN': 'PROPN',
  'TN': 'PROPN',
  'WN': 'PROPN',
  'AN': 'PROPN',
  'CN': 'PROPN',
  'FN': 'PROPN',
  'LN': 'PROPN',
  'ON': 'PROPN',
  'QN': 'PROPN',
  'YN': 'PROPN'},
 'posToFeatMapping': {'NOUN': ['Animacy', 'Case', 'Number'],
  'VERB': ['Aspect',
   'Mood',
   'Number',
   'Person',
   'Polarity',
   'Verb Form',
   'Voice'],
  'ADJ': ['Case', 'Number'],
  'NUM': ['Case'],
  'PROPN': ['Case', 'Number', 'Animacy']},
 'featureMapping': {'Case': {'ABS': 'Abs',
   'GEN': 'Gen',
   'ERG': 'Erg',
   'ABL': 'Abl'}

### adding tags to dict

In [344]:
# base_dix_path = './apertium-sux.sux.dix'
# mytree = ET.parse(base_dix_path)
# root = mytree.getroot()

In [345]:
old_alpha = root.find('sdefs')
root.remove(old_alpha)

In [346]:
symb_defs = ET.Element("sdefs")
root.append(symb_defs)
for tag in uniqe_tags:
    s1 = ET.SubElement(symb_defs,"sdef")
    s1.set('n',tag)
    # symb_defs.append(s1)

In [347]:
tree = ET.ElementTree(root)
ET.indent(tree, ' ')

In [348]:
sux_dix_name = "apertium-sux.sux.dix"
with open (sux_dix_name, "wb") as f :
        tree.write(f,encoding = "UTF-8", xml_declaration = True)

In [119]:
s = '''<prn><subj><p1><mf><sg>:-ga-e
<prn><subj><p1><mf><sg>:-ga2-e
<prn><subj><p1><mf><sg>:-ge26-e
<prn><subj><p2><mf><sg>:-za-e
<prn><subj><p2><mf><sg>:-ze2-e
<prn><subj><p3><mf><sg>:-e-ne
<prn><subj><p3><mf><sg>:-a-ne
<prn><subj><p3><nt><pl>:-e-ne-ne
<prn><subj><p3><nt><pl>:-a-ne-ne'''

In [128]:
temp = []
for i in s.split('\n'):
    temp.append(i.split(':')[1])

In [129]:
pd.DataFrame(temp)

,0
0,-ga-e
1,-ga2-e
2,-ge26-e
3,-za-e
4,-ze2-e
5,-e-ne
6,-a-ne
7,-e-ne-ne
8,-a-ne-ne


## Paradefs

In [349]:
para_defs = root.find('pardefs')

In [350]:
if para_defs:
    root.remove(para_defs)

In [351]:
extracted_sux_eng =  extracted_df[['filename','form','lem','eng_translation','tags']]
extracted_sux_eng.head()

,filename,form,lem,eng_translation,tags
0,P100723.conll,1(disz),1(disz),one,[num]
1,P100723.conll,udu-nita2,udunita,ram,"[n, abs]"
2,P100723.conll,ba-usz2,ba-usz,die,"[mid, vblex, p3, sg, s]"
3,P100723.conll,1(disz),1(disz),one,[num]
4,P100723.conll,sila4,sila,lamb,[n]


In [352]:
extracted_sux_eng.shape

(15394, 5)

In [353]:
extracted_sux_eng['tags_str'] = extracted_sux_eng['tags'].apply(lambda x:"_".join(x))

In [354]:
extracted_sux_eng

,filename,form,lem,eng_translation,tags,tags_str
0,P100723.conll,1(disz),1(disz),one,[num],num
1,P100723.conll,udu-nita2,udunita,ram,"[n, abs]",n_abs
2,P100723.conll,ba-usz2,ba-usz,die,"[mid, vblex, p3, sg, s]",mid_vblex_p3_sg_s
3,P100723.conll,1(disz),1(disz),one,[num],num
4,P100723.conll,sila4,sila,lamb,[n],n
...,...,...,...,...,...,...
27,P480072.conll,limmu2-ba,limmu,four,"[num, p3, sg, nh, poss, gen, abs]",num_p3_sg_nh_poss_gen_abs
28,P480072.conll,a-a-kal-la,Ayakala,Ayakala,[pn],pn
29,P480072.conll,ensi2,ensik,ruler,[n],n
30,P480072.conll,umma{ki},Umma,Umma,"[sn, gen, abs]",sn_gen_abs


In [355]:
# extracted_sux_eng_unique = extracted_sux_eng.drop_duplicates(subset=['form','tags_str'])
extracted_sux_eng_unique = extracted_sux_eng.drop_duplicates(subset=['form','tags_str'])
extracted_sux_eng_unique = extracted_sux_eng_unique[extracted_sux_eng_unique['tags_str']!='']
extracted_sux_eng_unique = extracted_sux_eng_unique[extracted_sux_eng_unique['tags_str']!='_']
extracted_sux_eng_unique.shape

(2918, 6)

In [356]:
extracted_sux_eng_unique_sort = extracted_sux_eng_unique.sort_values(['form']).reset_index(drop=True)
extracted_sux_eng_unique_sort

,filename,form,lem,eng_translation,tags,tags_str
0,P303478.conll,...-kam,_(_),_,"[num, gen, cop, p3, sg]",num_gen_cop_p3_sg
1,P142766.conll,1(asz),1(asz),one,[num],num
2,P429788.conll,1(asz)-a,1(disz),one,"[num, lp1]",num_lp1
3,P448068.conll,1(asz@c),1(asz@c),one,[num],num
4,P454156.conll,1(ban2),1(ban),unit,[n],n
...,...,...,...,...,...,...
2913,P122096.conll,{uruda}ha-bu3-da,habuda,hoe,[n],n
2914,P136937.conll,{uruda}ur2-zi-ir,urzir,tree_type,[n],n
2915,P416146.conll,{uruda}zi-ir,zir,slip,[n],n
2916,P478822.conll,|BAD3.AN|{ki}-ta,Der,Der,"[sn, abl]",sn_abl


In [357]:
extracted_sux_eng_unique_sort.tail(20)

,filename,form,lem,eng_translation,tags,tags_str
2898,P141530.conll,{gi}si2-im-tum,simtum,measuring_container,[n],n
2899,P144427.conll,{gi}su7-su7,susu,reed_type,[n],n
2900,P141675.conll,{kusz}du10-gan,duggan,leather_bag,[n],n
2901,P203987.conll,{kusz}du10-gan-na,duggan,leather_bag,"[n, gen, lp1]",n_gen_lp1
2902,P141675.conll,{kusz}e-sir2,esir,shoe,[n],n
2903,P142827.conll,{kusz}suhub2,suhub,boots,[n],n
2904,P100958.conll,{kusz}suhub2-sze3,suhub,boots,"[n, term]",n_term
2905,P141675.conll,{kusz}szu4-gan,szugan,pot_type,[n],n
2906,P141675.conll,{kusz}ummu3,ummud,waterskin,[n],n
2907,P101761.conll,{munus}asz2-gar3,aszgar,female_kid,[n],n


In [358]:
main_tags = ['n', 'vblex', 'aj', 'av', 'num', 'cnjcoo', 'det', 'j', 'pp']
NER_LIST = ['pn', 'on', 'fn', 'mn', 'sn', 'dn', 'gn', 'tn', 'wn', 'en', 'an', 'rn']
# tag_mapper = {'nu':'num', 'v':'vblex', 'cnj':'cnjcoo'}

In [359]:
primary_tag = []
secondary_tags = []

for i, row in extracted_sux_eng_unique_sort.iterrows():
    p_tg=''
    for j in main_tags:
        if j in row['tags']:
            p_tg = j
            break
    if p_tg=='' and len(row['tags'])>0:
        p_tg = row['tags'][0]
        if p_tg in NER_LIST:
            p_tg = 'n'

    
    secondary_tag_ls = list(set(row['tags'])-set([p_tg]))
    
    # add tags
    primary_tag.append(p_tg)
    secondary_tags.append(secondary_tag_ls)


In [360]:
extracted_sux_eng_unique_sort['primary_tag'] = primary_tag
extracted_sux_eng_unique_sort['secondary_tags'] = secondary_tags
extracted_sux_eng_unique_sort.head()

,filename,form,lem,eng_translation,tags,tags_str,primary_tag,secondary_tags
0,P303478.conll,...-kam,_(_),_,"[num, gen, cop, p3, sg]",num_gen_cop_p3_sg,num,"[sg, p3, gen, cop]"
1,P142766.conll,1(asz),1(asz),one,[num],num,num,[]
2,P429788.conll,1(asz)-a,1(disz),one,"[num, lp1]",num_lp1,num,[lp1]
3,P448068.conll,1(asz@c),1(asz@c),one,[num],num,num,[]
4,P454156.conll,1(ban2),1(ban),unit,[n],n,n,[]


In [361]:
extracted_sux_eng_unique_sort.tail(20)

,filename,form,lem,eng_translation,tags,tags_str,primary_tag,secondary_tags
2898,P141530.conll,{gi}si2-im-tum,simtum,measuring_container,[n],n,n,[]
2899,P144427.conll,{gi}su7-su7,susu,reed_type,[n],n,n,[]
2900,P141675.conll,{kusz}du10-gan,duggan,leather_bag,[n],n,n,[]
2901,P203987.conll,{kusz}du10-gan-na,duggan,leather_bag,"[n, gen, lp1]",n_gen_lp1,n,"[gen, lp1]"
2902,P141675.conll,{kusz}e-sir2,esir,shoe,[n],n,n,[]
2903,P142827.conll,{kusz}suhub2,suhub,boots,[n],n,n,[]
2904,P100958.conll,{kusz}suhub2-sze3,suhub,boots,"[n, term]",n_term,n,[term]
2905,P141675.conll,{kusz}szu4-gan,szugan,pot_type,[n],n,n,[]
2906,P141675.conll,{kusz}ummu3,ummud,waterskin,[n],n,n,[]
2907,P101761.conll,{munus}asz2-gar3,aszgar,female_kid,[n],n,n,[]


In [362]:
extracted_sux_eng_unique_sort['primary_tag'].unique()

array(['num', 'n', 'vblex', 'cnjcoo'], dtype=object)

In [363]:
# tree = ET.ElementTree(root)
# ET.indent(tree, ' ')

In [364]:
# sux_dix_name = "apertium-sux.sux.dix"
# with open (sux_dix_name, "wb") as f :
#         tree.write(f,encoding = "UTF-8", xml_declaration = True)

## Setions

In [365]:
secs = root.findall('section')

In [366]:
for sec in secs:
    root.remove(sec)

In [367]:
extracted_sux_eng_unique_sort.head()

,filename,form,lem,eng_translation,tags,tags_str,primary_tag,secondary_tags
0,P303478.conll,...-kam,_(_),_,"[num, gen, cop, p3, sg]",num_gen_cop_p3_sg,num,"[sg, p3, gen, cop]"
1,P142766.conll,1(asz),1(asz),one,[num],num,num,[]
2,P429788.conll,1(asz)-a,1(disz),one,"[num, lp1]",num_lp1,num,[lp1]
3,P448068.conll,1(asz@c),1(asz@c),one,[num],num,num,[]
4,P454156.conll,1(ban2),1(ban),unit,[n],n,n,[]


In [368]:
# tag_mapper = {'N':'n','NU':'num', 'V':'vblex', 'CNJ':'cnjcoo'}

In [369]:
section = ET.Element("section")
section.set('id',"main")
section.set('type',"standard")
root.append(section)

In [370]:
for i,row in extracted_sux_eng_unique_sort.iterrows():
    surface_form = row['form']
    lem = row['lem']
    tags = row['tags']
    p_tag = row['primary_tag']
    
    e = ET.SubElement(section,"e")
    p = ET.SubElement(e,"p")
    
    # left
    l = ET.SubElement(p,"l")
    l.text = surface_form
    
    # right
    r = ET.SubElement(p,"r")
    r.text = lem
    s = ET.SubElement(r,"s")
    if p_tag in uniqe_tags:
        s.set("n",p_tag)
    else:
        s.set("n","n")
        
    # for t in tags:
    #     if t in uniqe_tags:
    #         s = ET.SubElement(r,"s")
    #         mp_tag = t
    #         s.set("n",mp_tag)
    #     # r.append(s)

In [371]:
# uniqe_tags

In [372]:
extracted_sux_eng_unique_sort

,filename,form,lem,eng_translation,tags,tags_str,primary_tag,secondary_tags
0,P303478.conll,...-kam,_(_),_,"[num, gen, cop, p3, sg]",num_gen_cop_p3_sg,num,"[sg, p3, gen, cop]"
1,P142766.conll,1(asz),1(asz),one,[num],num,num,[]
2,P429788.conll,1(asz)-a,1(disz),one,"[num, lp1]",num_lp1,num,[lp1]
3,P448068.conll,1(asz@c),1(asz@c),one,[num],num,num,[]
4,P454156.conll,1(ban2),1(ban),unit,[n],n,n,[]
...,...,...,...,...,...,...,...,...
2913,P122096.conll,{uruda}ha-bu3-da,habuda,hoe,[n],n,n,[]
2914,P136937.conll,{uruda}ur2-zi-ir,urzir,tree_type,[n],n,n,[]
2915,P416146.conll,{uruda}zi-ir,zir,slip,[n],n,n,[]
2916,P478822.conll,|BAD3.AN|{ki}-ta,Der,Der,"[sn, abl]",sn_abl,n,"[sn, abl]"


In [373]:
tree = ET.ElementTree(root)
# ET.indent(tree, ' ')

In [374]:
tree_string_sux = ET.tostring(root,xml_declaration=True).decode("utf-8").replace('<e>','\n<e>').encode('utf-8')

In [375]:
# temp.encode('utf-8')

In [377]:
sux_dix_name = "apertium-sux.sux.dix"
with open (sux_dix_name, "wb") as f :
        f.write(tree_string_sux)

In [376]:
# sux_dix_name = "apertium-sux.sux.dix"
# with open (sux_dix_name, "wb") as f :
#         tree.write(f,encoding = "UTF-8", xml_declaration = True)

# Bi-Dictionary

In [222]:
extracted_sux_eng_unique_sort

,filename,form,lem,eng_translation,tags,tags_str,primary_tag,secondary_tags
0,P303478.conll,...-kam,_(_),_,"[num, gen, cop, p3, sg]",num_gen_cop_p3_sg,num,"[sg, p3, gen, cop]"
1,P142766.conll,1(asz),1(asz),one,[num],num,num,[]
2,P429788.conll,1(asz)-a,1(disz),one,"[num, lp1]",num_lp1,num,[lp1]
3,P448068.conll,1(asz@c),1(asz@c),one,[num],num,num,[]
4,P454156.conll,1(ban2),1(ban),unit,[n],n,n,[]
...,...,...,...,...,...,...,...,...
2913,P122096.conll,{uruda}ha-bu3-da,habuda,hoe,[n],n,n,[]
2914,P136937.conll,{uruda}ur2-zi-ir,urzir,tree_type,[n],n,n,[]
2915,P416146.conll,{uruda}zi-ir,zir,slip,[n],n,n,[]
2916,P478822.conll,|BAD3.AN|{ki}-ta,Der,Der,"[sn, abl]",sn_abl,n,"[sn, abl]"


In [240]:
sum_eng_dict_v1 = {}
for sux_lem,eng_trans in zip(extracted_df['lem'],extracted_df['eng_translation']):
    sum_eng_dict_v1[sux_lem] = eng_trans
print(len(sum_eng_dict_v1))
sum_eng_dict_v1

1679


{'1(disz)': 'one',
 'udunita': 'ram',
 'ba-usz': 'die',
 'sila': 'unit',
 'azur': 'type_of_sheep',
 'ki': 'place',
 'Uru': 'Uru',
 'szag': 'heart',
 'bala': 'turn',
 'kiszib': 'seal',
 'Arad': 'Arad',
 'iti': 'month',
 'Szesagku': 'Szesagku',
 'mu': 'year',
 'us': 'follow',
 'Kimasz': 'Kimasz',
 'ba-hulu': 'destroy',
 'Aradmu': 'Aradmu',
 'dubsar': 'scribe',
 'dumu': 'child',
 'Urnigar': 'Urnigar',
 'kusz': 'leather',
 'udu': 'sheep',
 'u': 'grass',
 'maszgal': 'billy_goat',
 'ba-ug': 'die',
 'ud': 'day',
 '1(u)': 'ten',
 'la': 'hang',
 '1(disz@t)': 'one',
 'Tummal': 'Tummal',
 'Endingirmu': 'Endingirmu',
 "Szulgi'irimu": "Szulgi'irimu",
 'szu': 'hand',
 '-teg': 'accept',
 'Akiti': 'Akiti',
 'en': 'priest',
 'Nanna': 'Nanna',
 'ba-hug': 'hire',
 '3(disz)': 'one',
 'gig': 'unit',
 '2(u)': 'ten',
 'sze': 'barley',
 'igi6(disz)': 'one',
 'Urbagara': 'Urbagara',
 '1(ban)': 'unit',
 'inun': 'ghee',
 '6(disz)': 'one',
 'ga': 'milk',
 'gazi': 'condiment',
 'Urmes': 'Urmes',
 'Urdumuzi': 'Urdu

In [284]:
base_dix_path = '../apertium-sux-eng/apertium-sux-eng.sux-eng.dix'
mytree = ET.parse(base_dix_path)
root = mytree.getroot()
root

<Element 'dictionary' at 0x10c38c450>

### adding tags to dict

In [285]:
# base_dix_path = './apertium-sux.sux.dix'
# mytree = ET.parse(base_dix_path)
# root = mytree.getroot()

In [286]:
old_alpha = root.find('sdefs')
root.remove(old_alpha)

In [287]:
symb_defs = ET.Element("sdefs")
root.append(symb_defs)
for tag in uniqe_tags:
    s1 = ET.SubElement(symb_defs,"sdef")
    s1.set('n',tag)
    # symb_defs.append(s1)

In [288]:
tree = ET.ElementTree(root)
ET.indent(tree, ' ')

In [289]:
# sux_dix_name = "apertium-sux.sux.dix"
# with open (sux_dix_name, "wb") as f :
#         tree.write(f,encoding = "UTF-8", xml_declaration = True)

In [290]:
# old_alpha = root.find('alphabet')
# root.remove(old_alpha)

In [291]:
# el_alpha = ET.Element("alphabet")
# el_alpha.text = alphs_string

In [292]:
# root.append(el_alpha)

In [293]:
# tree = ET.ElementTree(root)
# # ET.indent(tree, space="\t", level=0)

## ==========================

## Sections

In [294]:
secs = root.findall('section')
for sec in secs:
    root.remove(sec)

In [295]:
section = ET.Element("section")
section.set('id',"main")
section.set('type',"standard")
root.append(section)

In [296]:
for i,row in extracted_sux_eng_unique_sort.iterrows():
    surface_form = row['form']
    lem = row['lem']
    tags = row['tags']
    eng_translation = row['eng_translation']
    p_tag = row['primary_tag']
    
    e = ET.SubElement(section,"e")
    p = ET.SubElement(e,"p")
    
    # left
    l = ET.SubElement(p,"l")
    l.text = lem
    s = ET.SubElement(l,"s")
    if p_tag in uniqe_tags:
        s.set("n",p_tag)
    else:
        s.set("n","n")



    # for t in tags:
    #     if t in uniqe_tags:
    #         s = ET.SubElement(l,"s")
    #         mp_tag = t
    #         try:
    #             mp_tag = tag_mapper[t]
    #         except:
    #             pass
    #         s.set("n",mp_tag)


    # right
    r = ET.SubElement(p,"r")
    r.text = eng_translation
    s = ET.SubElement(r,"s")
    if p_tag in uniqe_tags:
        s.set("n",p_tag)
    else:
        s.set("n","n")




    # for t in tags:

    #     mp_tag = t
    #     try:
    #         mp_tag = tag_mapper[t]
    #     except:
    #         pass
    # s.set("n","n")
        # r.append(s)

In [297]:
tree = ET.ElementTree(root)
# ET.indent(tree, ' ')

In [298]:
sux_dix_name = "../apertium-sux-eng/apertium-sux-eng.sux-eng.dix"
with open (sux_dix_name, "wb") as f :
        tree.write(f,encoding = "UTF-8", xml_declaration = True)

In [317]:
temp = ET.tostring(root,xml_declaration=True).decode("utf-8").replace('<e>','\n<e>').encode('utf-8')

In [318]:
# temp.encode('utf-8')

In [319]:
sux_dix_name = "../apertium-sux-eng/apertium-sux-eng.sux-eng.dix"
with open (sux_dix_name, "wb") as f :
        f.write(temp)